# Update a Feature Layer View with a Polygon Area of Interest
This notebook will update the `viewLayerDefinition` of a hosted feature layer view to only show features that interset a polygon. In our example below, we are querying a layer of generalized U.S. States for boundary of Indiana, then applying that polygon as the spatial filter for another layer.

**Please note this may have negative performance implications. Consider your polygons and the level of detail they have before implementing this approach.**

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry, SpatialReference

from copy import deepcopy
gis = GIS("home")

Some basic setup of what state boundary we want to grab for our filter

In [ ]:
state_abbr_to_use = 'IN'
state_abbr_field = 'STATE_ABBR'
where_clause = f"{state_abbr_field} = '{state_abbr_to_use}'"
where_clause

Get a reference to the State boundary layer and the hosted feature layer view we are going to update

In [ ]:
fl = FeatureLayer.fromitem(gis.content.get('99fd67933e754a1181cc755146be21ca'))
flview_to_update = FeatureLayer.fromitem(gis.content.get('400ab9c2d7024058b5c6c2f38a714fd3'))

Here is our template JSON we will use. We'll replace the `rings` property with what comes back from our State boundary query

In [ ]:
vld_base = {
  "viewLayerDefinition": {
    "filter": {
      "operator": "esriSpatialRelIntersects",
      "value": {
        "geometryType": "esriGeometryPolygon",
        "geometry": {
          "rings": [
            [
              [-13478878.103229811, 5474302.767027485],
              [-12940761.424102314, 5880336.2612782335],
              [-12877165.816569064, 5469410.797217235],
              [-13718584.62393206, 4857914.570935988],
              [-13713692.654121809, 5430275.038735235],
              [-13478878.103229811, 5474302.767027485]
            ]
          ],
          "spatialReference": { "wkid": 102100, "latestWkid": 3857 }
        }
      }
    }
  }
}

Make a copy of our template JSON

In [ ]:
vld_to_update = deepcopy(vld_base)

Execute our query Indiana's geometry

In [ ]:
fset = fl.query(where=where_clause, out_fields=state_abbr_field)
geom = fset.features[0].geometry

Set our `rings` to be that of Indiana's

In [ ]:
vld_to_update['viewLayerDefinition']['filter']['value']['geometry']['rings'] = geom['rings']

Tell the feature layer view to update its definition

In [ ]:
flview_to_update.manager.update_definition(vld_to_update)